In [ ]:
# -*- coding: utf-8 -*-
# @Time: 2022/2/20 11:58
# @Author: Gonghao Zhang
# @Email: hewittzgh@gmail.com
# @File: DAPplace.ipynb
# @Software: JupyterLab

In [ ]:
import math
import copy
import time
import random
import numpy as np
import networkx as nx
import gurobipy as gp
from gurobipy import GRB
from scipy import spatial
from sklearn import cluster
import matplotlib.pyplot as plt
from networkx.algorithms import bipartite
import sklearn_extra.cluster as ex_cluster
from sklearn.metrics import pairwise_distances

In [ ]:
# define the decorator decorator to calculate the running time of the function
def decorator_with_param(name):
    def run_time(func):
        def warp(*args, **kwargs):
            t_start = time.perf_counter()
            ret = func(*args, **kwargs)
            t_end = time.perf_counter()
            print("[INFO] " + name + " function run time: " + "%f" % (t_end - t_start))
            return ret
        return warp
    return run_time

In [ ]:
class HeuristicApproach(object):
    """HeuristicApproach class implements a heuristic optimization method for DAP placement
    Attributes:
        rho: density of smart meter                                default = 2000              (SMs/km^2)              [float]
        A: area                                                    default = 4                 (km^2)                  [float]
        N_POLE: the number of Poles                                default = 2500                                      [int]
        N_MAX_SP: the maximal number of SMs per DAP                default = 2000                                      [int]
        N_MAX_SS: the maximal number of SMs per relay SM           default = 10                                        [int]
        D_MAX: maximum communication distance                      default = 100               (m)                     [float]
        a: unit price of DAP                                       default = 2000              (dollar)                [float]
        b: the transmission cost for one SM per unit path loss     default = 2.53e-10          (dollar)                [float]
        c: delay cost per unit path                                default = 5                 (dollar)                [float]
        PL0: the initial expected path loss over distance dij      default = 43                (dB)                    [float]
        d0: unit distance                                          default = 1                 (m)                     [float]
        alpha: PL proportional power                               default = 3                                         [float]
        DRAW_CONTROL: drawing or not                               default = False                                     [bool]
        DRAW_NODE_SIZE: node size to draw                          default = 1                                         [float]
        DRAW_EDGE_WIDTH: edge width to draw                        default = 0.01                                      [float]
        ILP_INFO: printing ILP info or not                         default = False                                     [bool]
    """
    
    def __init__(self, 
                 rho = 2000, A = 4, N_POLE = 2500, N_MAX_SP = 2000, N_MAX_SS = 5, D_MAX = 200, 
                 a = 2000, b = 2.53e-10, c = 5, PL0 = 43, d0 = 1, alpha = 3, 
                 DRAW_CONTROL = False, DRAW_NODE_SIZE = 1, DRAW_EDGE_WIDTH = 0.01, ILP_INFO = False):
        """Inits HeuristicApproach class with preset value."""
        
        # define parameter
        self.PARAM_rho              = rho                                         # density of smart meter
        self.PARAM_A                = A                                           # area
        self.PARAM_N_POLE           = N_POLE                                      # the number of Poles
        self.PARAM_N_MAX_SP         = N_MAX_SP                                    # the maximal number of SMs per DAP
        self.PARAM_N_MAX_SS         = N_MAX_SS                                    # the maximal number of SMs per relay SM
        self.PARAM_D_MAX            = D_MAX                                       # maximum communication distanc
        self.PARAM_a                = a                                           # unit price of DAP
        self.PARAM_b                = b                                           # the transmission cost for one SM per unit path loss
        self.PARAM_c                = c                                           # delay cost per unit path
        self.PARAM_PL0              = PL0                                         # the initial expected path loss over distance dij
        self.PARAM_d0               = d0                                          # unit distance
        self.PARAM_alpha            = alpha                                       # PL proportional power

        self.DRAW_CONTROL           = DRAW_CONTROL                                # drawing or not
        self.DRAW_NODE_SIZE         = DRAW_NODE_SIZE                              # node size to draw
        self.DRAW_EDGE_WIDTH        = DRAW_EDGE_WIDTH                             # edge width to draw
        self.ILP_INFO               = ILP_INFO                                    # print ILP info or not
        
        self.N_SM                   = int(self.PARAM_rho * self.PARAM_A)          # the number of SMs
        self.AREA_RANGE             = 1000 * math.sqrt(self.PARAM_A)              # m
        self.RETURN_STEP1           = False                                       # indicates whether the first step needs to be returned
        
        # define variables
        self.sm_nodes_index                    = []                               # smart meters nodes index
        self.sm_nodes_pos_list                 = []                               # smart meters nodes position list
        self.sm_nodes_pos                      = {}                               # smart meters nodes position
        self.pole_nodes_index                  = []                               # pole nodes index
        self.pole_nodes_pos_list               = []                               # pole nodes position list
        self.pole_nodes_pos                    = {}                               # pole nodes position
        
    def InitializeCleanEnvironment(self):
        """Initialize a clean running environment."""
        
        self.K                      = math.ceil(self.N_SM / self.PARAM_N_MAX_SP)  # the number of initial clusters
        self.RETURN_STEP1           = False                                       # indicates whether the first step needs to be returned
        
        # define variables
        self.old_cluster_centers_index         = []                               # old clustering centers index
        self.new_cluster_centers_index         = []                               # new clustering centers index
        self.cluster_centers_pos_list          = []                               # clustering centers position list
        self.cluster_center_sm_edges           = []                               # old clustering center -> its smart meter edge
        self.old_bipartite_center_pole_edges   = []                               # old edge of cluster center pointing to pole in bipartite
        self.new_bipartite_center_pole_edges   = []                               # new edge of cluster center pointing to pole in bipartite
        self.old_selected_pole_index           = []                               # old pole selected to place DAP index
        self.new_selected_pole_index           = []                               # new pole selected to place DAP index
        self.old_selected_pole_pos             = {}                               # old pole selected to place DAP position
        self.new_selected_pole_pos             = {}                               # new pole selected to place DAP position
        
        if self.DRAW_CONTROL:
            
            self.cluster_centers_pos                      = {}                        # clustering centers position
            self.old_cluster_sm_nodes_color               = []                        # old smart meter node color belonging to different cluster
            self.new_cluster_sm_nodes_color               = []                        # new smart meter node color belonging to different cluster
            self.old_bipartite_cluster_centers_pos        = {}                        # old cluster centers position in bipartite layout
            self.new_bipartite_cluster_centers_pos        = {}                        # new cluster centers position in bipartite layout
            self.old_bipartite_poles_pos                  = {}                        # old poles position in bipartite layout
            self.new_bipartite_poles_pos                  = {}                        # new poles position in bipartite layout
            self.old_bipartite_match_poles_pos            = {}                        # old matched poles position in bipartite layout
            self.new_bipartite_match_poles_pos            = {}                        # new matched poles position in bipartite layout
            self.old_bitpartite_match_edges               = []                        # old cluster centers to matched poles edges
            self.new_bitpartite_match_edges               = []                        # new cluster centers to matched poles edges
            
        self.tol                               = 1e-4                             # convergence tolerance
        self.old_cost                          = float('inf')                     # K cost
        self.new_cost                          = 0                                # K + 1 cost
        self.old_installation_cost             = 0                                # K installation cost
        self.new_installation_cost             = 0                                # K + 1 installation cost
        self.old_routing_cost                  = 0                                # K routing cost
        self.new_routing_cost                  = 0                                # K + 1 routing cost   
        self.old_routing                       = {}                               # K routing
        self.new_routing                       = {}                               # K + 1 routing
        self.Iter                              = 0                                # the number of iterations
        self.Iter_index                        = []                               # with the iteration of index
        self.Iter_installation_cost            = []                               # with the iteration of installation cost
        self.Iter_transmission_cost            = []                               # with the iteration of transmission cost
        self.Iter_delay_cost                   = []                               # with the iteration of delay cost
        
        # define Graphs for operating
        self.G = nx.DiGraph()                                                      # overall graph
        self.B = nx.DiGraph()                                                      # bipartite graph for minimum weight matching
        
        # setting the gurobipy ILP environment
        self.env = gp.Env(empty = True)
        self.env.setParam('OutputFlag', self.ILP_INFO)
        self.env.start()
        
    def ManualChangeParameter(self, N_MAX_SS = None, D_MAX = None, c = None, alpha = None):
        """Manual Change parameters to compare results.
        Attributes:
            N_MAX_SS
            D_MAX
            c
            alpha
        """
        
        if N_MAX_SS != None:
            self.PARAM_N_MAX_SS = N_MAX_SS
            
        if D_MAX != None:
            self.PARAM_D_MAX = D_MAX
            
        if c != None:
            self.PARAM_c = c
            
        if alpha != None:
            self.PARAM_alpha = alpha
        
    def GetSimData(self):
        """Generates simulation data."""
        
        self.sm_nodes_index.clear()
        self.sm_nodes_pos_list.clear()
        self.sm_nodes_pos.clear()
        self.pole_nodes_index.clear()
        self.pole_nodes_pos_list.clear()
        self.pole_nodes_pos.clear()
        
        sm_coor   = np.random.randint(- self.AREA_RANGE / 2, self.AREA_RANGE / 2 + 1, size = (self.N_SM, 2))
        pole_coor = np.random.randint(- self.AREA_RANGE / 2, self.AREA_RANGE / 2 + 1, size = (self.PARAM_N_POLE, 2))
        
        for index, pos in enumerate(sm_coor):
            sm_name = 's' + str(index + 1)
            sm_pos  = pos.tolist()
            self.sm_nodes_index.append(sm_name)
            self.sm_nodes_pos_list.append(sm_pos)
            self.sm_nodes_pos[sm_name] = sm_pos
            
        for index, pos in enumerate(pole_coor):
            pole_name = 'p' + str(index + 1)
            pole_pos  = pos.tolist()
            self.pole_nodes_index.append(pole_name)
            self.pole_nodes_pos_list.append(pole_pos)
            self.pole_nodes_pos[pole_name] = pole_pos
        
    def DisCutoff(self, x, y):
        """Cutoff distance compute for pairwise_distances."""
        M = math.pow(self.AREA_RANGE, 10) # very large number
        X = np.vstack([x, y])
        dis = spatial.distance.pdist(X)
        if dis > self.PARAM_D_MAX:
            dis = M * dis
        return dis
    
    def DisEnergy(self, x, y):
        """Energy distance compute for pairwise_distances."""
        X = np.vstack([x, y])
        dis = spatial.distance.pdist(X)
        d = self.PARAM_b * self.PL(dis)
        return d
    
    def DisCutoffEnergy(self, x, y):
        """Cutoff-Energy distance compute for pairwise_distances."""
        M = math.pow(self.AREA_RANGE, 10) # very large number
        X = np.vstack([x, y])
        dis = spatial.distance.pdist(X)
        d = self.PARAM_b * self.PL(dis)
        if dis > self.PARAM_D_MAX:
            d = M * d
        return d
            
    @decorator_with_param('KmeansClustering')
    def KmeansClustering(self, cluster_data):
        """KMeans clustering method.
        Attributes:
            cluster_data: data for KMeans clustering                          [np.array]
        Returns:
            labels: clustering labels                                         [np.array]
            centers: clustering centers                                       [np.array]
        """
        
        kmeans = cluster.KMeans(n_clusters = self.K, random_state = 0).fit(cluster_data)
        return kmeans.labels_, kmeans.cluster_centers_
    
    @decorator_with_param('MiniBatchKmeansClustering')
    def MiniBatchKmeansClustering(self, cluster_data, batch_size = 1024):
        """Mini-Batch KMeans clustering method.
        Attributes:
            cluster_data: data for mini-batch KMeans clustering               [np.array]
            batch_size: size of the mini batches
                        default: 1024
                        recommend: batch_size >= 256 * number of cores        [int]
        Returns:
            labels: clustering labels                                         [np.array]
            centers: clustering centers                                       [np.array]
        """
        
        minibatchkmeans = cluster.MiniBatchKMeans(n_clusters = self.K, random_state = 0, batch_size = batch_size).fit(cluster_data)
        return minibatchkmeans.labels_, minibatchkmeans.cluster_centers_
    
    @decorator_with_param('KMedoidsClustering')
    def KMedoidsClustering(self, distance_matrix):
        """KMedoids clustering method.
        Attributes:
            distance_matrix: precomputed distance matrix                      [np.ndarray]
        Returns:
            labels: clustering labels                                         [np.array]
            centers: clustering centers                                       [np.array]
        """
        
        kmedoids = ex_cluster.KMedoids(n_clusters = self.K, metric = 'precomputed', random_state = 0).fit(distance_matrix)
        
        cluster_centers_ = []
        for k in range(self.K):
            k_index = [index for index, label in enumerate(kmedoids.labels_) if label == k]
            k_pos_x = 0
            k_pos_y = 0
            for i in k_index:
                k_pos_x += self.sm_nodes_pos_list[i][0]
                k_pos_y += self.sm_nodes_pos_list[i][1]
            k_pos_x = k_pos_x / len(k_index)
            k_pos_y = k_pos_y / len(k_index)
            cluster_centers_.append(np.array([k_pos_x, k_pos_y]))
            
        return kmedoids.labels_, cluster_centers_
    
    @decorator_with_param('SpectralClustering')
    def SpectralClustering(self, distance_matrix):
        """Spectral clustering method.
        Attributes:
            distance_matrix: precomputed distance matrix                      [np.ndarray]
        Returns:
            labels: clustering labels                                         [np.array]
            centers: clustering centers                                       [np.array]
        """
        
        spectral = cluster.SpectralClustering(n_clusters = self.K, random_state = 0, affinity = 'precomputed').fit(distance_matrix)
        
        cluster_centers_ = []
        for k in range(self.K):
            k_index = [index for index, label in enumerate(spectral.labels_) if label == k]
            k_pos_x = 0
            k_pos_y = 0
            for i in k_index:
                k_pos_x += self.sm_nodes_pos_list[i][0]
                k_pos_y += self.sm_nodes_pos_list[i][1]
            k_pos_x = k_pos_x / len(k_index)
            k_pos_y = k_pos_y / len(k_index)
            cluster_centers_.append(np.array([k_pos_x, k_pos_y]))
            
        return spectral.labels_, cluster_centers_
    
    @decorator_with_param('AgglomerativeClustering')
    def AgglomerativeClustering(self, distance_matrix):
        """Agglomerative clustering method.
        Attributes:
            distance_matrix: precomputed distance matrix                      [np.ndarray]
        Returns:
            labels: clustering labels                                         [np.array]
            centers: clustering centers                                       [np.array]
        """
        
        agglomerative = cluster.AgglomerativeClustering(n_clusters = self.K, affinity = 'precomputed', linkage = 'complete').fit(distance_matrix)
        
        cluster_centers_ = []
        for k in range(self.K):
            k_index = [index for index, label in enumerate(agglomerative.labels_) if label == k]
            k_pos_x = 0
            k_pos_y = 0
            for i in k_index:
                k_pos_x += self.sm_nodes_pos_list[i][0]
                k_pos_y += self.sm_nodes_pos_list[i][1]
            k_pos_x = k_pos_x / len(k_index)
            k_pos_y = k_pos_y / len(k_index)
            cluster_centers_.append(np.array([k_pos_x, k_pos_y]))
            
        return agglomerative.labels_, cluster_centers_
    
    def PL(self, distance):
        """Compute PL value of dij.
        Attributes:
            distance: the distance between node i and node j                  [float]
        Returns:
            plvalue: the PL value of dij                                      [float]
        """
        
        plvalue = self.PARAM_PL0 * math.pow(distance / self.PARAM_d0, self.PARAM_alpha)
        return plvalue
    
    @decorator_with_param('MiniWeightMatching')
    def MiniWeightMatching(self, weight='weight'):
        """Minimum weight matching.
        Attributes:
            weight: the edge data key used to provide each value
                    default: 'weight'                                         [string]
        Returns:
            mwm: minimum weight matching result                               [python.dictionary]
        """
        
        mwfm = bipartite.minimum_weight_full_matching(self.B, weight=weight)
        return mwfm
    
    @decorator_with_param("ILP")
    def ILP(self, edges, Beta, nodes_index, source_node, target_node):
        """Integer linear programming.
        Attributes:
            edges: indices for accessing the graph edges variables            [gurobipy.tuplelist]
            Beta: objective coefficient(s) for graph edges variables          [gurobipy.tupledict]
            nodes_index: smart meter nodes index in graph                     [python.list]
        Returns:
            routing: routing path                                             [python.dictionary]
            routing_cost: routing cost                                        [float]
        """
        
        routing = {}
        routing_cost = 0
        
        # Create optimization model
        m = gp.Model(env = self.env, name = 'netflow')

        # Create variables
        flow = m.addVars(edges, lb = 0, obj = Beta, vtype = GRB.INTEGER, name = "flow")

        # constraints 26
        m.addConstrs((flow[i, j] == 1 for i, j in edges.select(source_node, '*')), "constraints_26")

        # Flow-conservation constraints 27
        m.addConstrs((flow.sum(i, '*') == flow.sum('*', i) for i in nodes_index), "constraints_27")

        # constraints 28
        m.addConstr(flow.sum('*', target_node) == len(nodes_index), "constraints_28")

        # constraints 29
        m.addConstrs((flow.sum('*', i) <= self.PARAM_N_MAX_SS for i in nodes_index), "constraints_29")

        # Compute optimal solution
        m.optimize()
        
        if m.Status == GRB.INFEASIBLE:
            self.K += 1
            m.dispose()
            self.RETURN_STEP1 = True
        elif m.Status == GRB.OPTIMAL:
            routing_cost = m.ObjVal
            routing = m.getAttr('X', flow)
            m.dispose()
            self.RETURN_STEP1 = False

        return routing, routing_cost
    
    def ClearILPEnv(self):
        """Clear ILP environment."""
        self.env.dispose()
    
    def OurHeuristic(self, clustering_type = 'kmeans', custom_metric = 'eu'):
        """The concrete implementation of our Heuristic Approach.
        Attributes:
            clustering_type: clustering algorithm used
                             default: 'kmeans'                                 
                             support: 'kmeans' 'minibatchkmeans' 'kmedoids' 'spectral' 'agglomerative'     [string]
            custom_metric: custom metrics, including 'eu', 'en', 'ct', 'cte'
                            'eu': euclidean
                            'en': energy
                            'ct': cutoff
                            'cte': cutoff-energy                                                           [string]
        """
        
        if clustering_type == 'kmeans' or clustering_type == 'minibatchkmeans':
            if custom_metric != 'eu':
                print("[ERROR] " + "'kmeans' or 'minibatchkmeans' does not support this input distance type.")
                return -1
        elif clustering_type == 'kmedoids' or clustering_type == 'spectral' or clustering_type =='agglomerative':
            if custom_metric == 'eu':
                print("[INFO] generating distance matrix for", clustering_type, "and", custom_metric, "...")
                distance_matrix = pairwise_distances(np.array(self.sm_nodes_pos_list), metric = 'euclidean')

            elif custom_metric == 'en':
                print("[INFO] generating distance matrix for", clustering_type, "and", custom_metric, "...")
                distance_matrix = pairwise_distances(np.array(self.sm_nodes_pos_list), metric = self.DisEnergy)

            elif custom_metric == 'ct':
                print("[INFO] generating distance matrix for", clustering_type, "and", custom_metric, "...")
                distance_matrix = pairwise_distances(np.array(self.sm_nodes_pos_list), metric = self.DisCutoff)

            elif custom_metric == 'cte':
                print("[INFO] generating distance matrix for", clustering_type, "and", custom_metric, "...")
                distance_matrix = pairwise_distances(np.array(self.sm_nodes_pos_list), metric = self.DisCutoffEnergy)

            else:
                print("[ERROR] " + "'kmedoids' or 'spectral' or 'agglomerative' does not support this input distance type.")
                return -1
        else:
            print("[ERROR] " + "the set clustering method is not supported.")
            return -1

        self.G.add_nodes_from(self.sm_nodes_index)
        self.G.add_nodes_from(self.pole_nodes_index)
        
        if self.DRAW_CONTROL:
            
            %matplotlib qt5
            # Set subplots and axes
            fig, ax = plt.subplots(2, 2, figsize=(20, 20))
            plt.subplots_adjust(left=0.05, bottom=0.05, top=0.95, right=0.95, hspace=0.2, wspace=0.1)
            ax[0][0].set_title("Smart Meters and Poles Distribution Map")
            ax[0][1].set_title("Heuristic Approach Visualization")
            ax[1][0].set_title("Assign Cluster Centers Bipartite")
            ax[1][1].set_title("Cost Optimization and Contributions")
            
        if self.DRAW_CONTROL:
            
            # set canvas and axis
            ax[0][0].set_xlim([-self.AREA_RANGE/2 - self.AREA_RANGE/30, self.AREA_RANGE/2 + self.AREA_RANGE/30])
            ax[0][0].set_ylim([-self.AREA_RANGE/2 - self.AREA_RANGE/30, self.AREA_RANGE/2 + self.AREA_RANGE/30])
            
            # set drawing options for the graph
            sm_nodes_draw_options = {
                "ax":            ax[0][0], 
                "pos":           self.sm_nodes_pos, 
                "nodelist":      self.sm_nodes_index, 
                "node_color":    "blue", 
                "node_size":     self.DRAW_NODE_SIZE, 
                "node_shape":    'o',
                "label":         "Smart Meters Nodes"
            }
            pole_nodes_draw_options = {
                "ax":            ax[0][0], 
                "pos":           self.pole_nodes_pos, 
                "nodelist":      self.pole_nodes_index, 
                "node_color":    "red", 
                "node_size":     self.DRAW_NODE_SIZE, 
                "node_shape":    '^', 
                "label":         "Pole Nodes"
            }
            
            # Draw
            nx.draw_networkx_nodes(self.G, **sm_nodes_draw_options)
            nx.draw_networkx_nodes(self.G, **pole_nodes_draw_options)
            
            # Draw Settings
            ax[0][0].legend(loc = 'lower right')
            ax[0][0].set_title("Smart Meters and Poles Distribution Map")
            ax[0][0].tick_params(left = True, bottom = True, labelleft = True, labelbottom = True)
            
            plt.pause(0.1)
        
        while True:
            
            print("[ITERATION] " + "Start iteration " + str(self.Iter + 1) + " ... ")
            
            # Step 1: cluster dataset with K centers
            
            if clustering_type == 'kmeans':
                cluster_labels, cluster_centers = self.KmeansClustering(np.array(self.sm_nodes_pos_list))
                
            elif clustering_type == 'minibatchkmeans':
                cluster_labels, cluster_centers = self.MiniBatchKmeansClustering(np.array(self.sm_nodes_pos_list))
                
            elif clustering_type == 'kmedoids':
                cluster_labels, cluster_centers = self.KMedoidsClustering(distance_matrix)
            
            elif clustering_type == 'spectral':
                cluster_labels, cluster_centers = self.SpectralClustering(distance_matrix)
            
            elif clustering_type == 'agglomerative':
                cluster_labels, cluster_centers = self.AgglomerativeClustering(distance_matrix)
                
            # analysis of clustering results
            self.new_cluster_centers_index.clear()
            self.cluster_centers_pos_list.clear()
            if self.DRAW_CONTROL:
                self.cluster_centers_pos.clear()
            
            for index, pos in enumerate(cluster_centers):
                cluster_center_name = 'c' + str(index + 1)
                cluster_center_pos  = pos.tolist()
                self.new_cluster_centers_index.append(cluster_center_name)
                self.cluster_centers_pos_list.append(cluster_center_pos)
                if self.DRAW_CONTROL:
                    self.cluster_centers_pos[cluster_center_name] = cluster_center_pos
                
            self.cluster_center_sm_edges.clear()
            for index, label in enumerate(cluster_labels):
                self.cluster_center_sm_edges.append(('c' + str(label + 1), 's' + str(index + 1)))
                
            self.G.add_nodes_from(self.new_cluster_centers_index)
            self.G.add_edges_from(self.cluster_center_sm_edges)
            
            if self.DRAW_CONTROL:
                
                # defines the function to get the cluster color map (hexadecimal)
                gen_colors = lambda n: list(map(lambda i: "#" + "%06x" % random.randint(0, 0xFFFFFF), range(n)))
                color_map = gen_colors(self.K) # for example, return ['#8fa420', '#82bdeb', '#7f3c17', '#bcf4c4']
                
                self.new_cluster_sm_nodes_color.clear()
                for i in cluster_labels:
                    self.new_cluster_sm_nodes_color.append(color_map[i])
                    
                ax[0][1].cla()
                # set canvas and axis
                ax[0][1].set_xlim([-self.AREA_RANGE/2 - self.AREA_RANGE/30, self.AREA_RANGE/2 + self.AREA_RANGE/30])
                ax[0][1].set_ylim([-self.AREA_RANGE/2 - self.AREA_RANGE/30, self.AREA_RANGE/2 + self.AREA_RANGE/30])
            
                # set drawing options for the graph
                cluster_sm_nodes_draw_options = {
                    "ax":            ax[0][1], 
                    "pos":           self.sm_nodes_pos, 
                    "nodelist":      self.sm_nodes_index, 
                    "node_size":     self.DRAW_NODE_SIZE, 
                    "node_color":    self.new_cluster_sm_nodes_color, 
                    "node_shape":    'o', 
                    "label":         "Smart Meters of Different Clusters"
                }
                cluster_centers_draw_options = {
                    "ax":            ax[0][1], 
                    "pos":           self.cluster_centers_pos, 
                    "nodelist":      self.new_cluster_centers_index, 
                    "node_size":     10 * self.DRAW_NODE_SIZE, 
                    "node_color":    "red", 
                    "node_shape":    'o', 
                    "label":         "Clustering Centers"
                }
                cluster_center_sm_edges_draw_options = {
                    "ax":            ax[0][1], 
                    "pos":           dict(**self.cluster_centers_pos, **self.sm_nodes_pos), 
                    "edgelist":      self.cluster_center_sm_edges, 
                    "width":         2 * self.DRAW_EDGE_WIDTH, 
                    "edge_color":    "black", 
                    "style":         'solid', 
                    "arrows":        False, 
                    "label":         "Smart Meters to Clustering Center Edges"
                }
                
                # Draw
                nx.draw_networkx_nodes(self.G, **cluster_sm_nodes_draw_options)
                nx.draw_networkx_nodes(self.G, **cluster_centers_draw_options)
                nx.draw_networkx_edges(self.G, **cluster_center_sm_edges_draw_options)
                
                # Draw Settings
                ax[0][1].legend(loc = 'lower right')
                ax[0][1].set_title("Heuristic Approach Visualization")
                ax[0][1].text(0.01, 0.95, "K = " + str(self.K), transform=ax[0][1].transAxes, 
                              fontdict={'size': '10', 'color': 'black', 'weight': 'bold'})
                ax[0][1].tick_params(left = True, bottom = True, labelleft = True, labelbottom = True)

                # 0.1s pause to display
                plt.pause(0.1)
            
            # constraint 15
            for cc in self.new_cluster_centers_index:
                if self.G.out_degree(cc) > self.PARAM_N_MAX_SP:
                    # constraint 15 violated
                    self.K += 1
                    self.RETURN_STEP1 = True
                    break
                    
            if self.RETURN_STEP1:
                self.RETURN_STEP1 = False
                self.G.remove_edges_from(self.cluster_center_sm_edges)
                self.G.remove_nodes_from(self.new_cluster_centers_index)
                continue
                
            # Step 2: assign clustering centers to poles
            disM = spatial.distance_matrix(self.cluster_centers_pos_list, self.pole_nodes_pos_list)
            self.new_bipartite_center_pole_edges.clear()
            for i, c_name in enumerate(self.new_cluster_centers_index):
                for j, p_name in enumerate(self.pole_nodes_index):
                    w = (disM[[i], [j]])[0]
                    self.new_bipartite_center_pole_edges.append((c_name, p_name, {'weight': w}))
                    
            self.B.add_nodes_from(self.new_cluster_centers_index, bipartite = 0)
            self.B.add_nodes_from(self.pole_nodes_index, bipartite = 1)
            self.B.add_edges_from(self.new_bipartite_center_pole_edges)
            
            # minimum weight matching
            mwfmresult = self.MiniWeightMatching(weight='weight')
            mwmresult  = dict([(key, mwfmresult[key]) for key in self.new_cluster_centers_index])
            self.new_selected_pole_index  = [m for m in mwmresult.values()]
            
            self.new_selected_pole_pos.clear()
            for i in self.new_selected_pole_index:
                self.new_selected_pole_pos[i] = self.pole_nodes_pos[i]
            
            if self.DRAW_CONTROL:
                
                bottom_nodes, top_nodes = bipartite.sets(self.B)
                
                bitpartite_nodes_pos    = nx.bipartite_layout(self.B, bottom_nodes)
                
                self.new_bipartite_cluster_centers_pos = dict([(key, bitpartite_nodes_pos[key]) for key in bottom_nodes])
                self.new_bipartite_poles_pos           = dict([(key, bitpartite_nodes_pos[key]) for key in top_nodes])
                self.new_bipartite_match_poles_pos     = dict([(key, bitpartite_nodes_pos[key]) for key in mwmresult.values()])
                
                self.new_bitpartite_match_edges.clear()
                for i in mwmresult.keys():
                    self.new_bitpartite_match_edges.append((i, mwmresult[i]))
                    
                # Set drawing options for the graph
                bipartite_cluster_centers_draw_options = {
                    "ax": ax[1][0], 
                    "pos": self.new_bipartite_cluster_centers_pos, 
                    "nodelist": bottom_nodes, 
                    "node_size": 10 * self.DRAW_NODE_SIZE, 
                    "node_color": "red", 
                    "node_shape": 'o', 
                    "label": "Clustering Center Node"
                }
                bipartite_poles_draw_options = {
                    "ax": ax[1][0], 
                    "pos": self.new_bipartite_poles_pos, 
                    "nodelist": top_nodes, 
                    "node_size": self.DRAW_NODE_SIZE, 
                    "node_color": "red", 
                    "node_shape": '^', 
                    "label": "Pole Node"
                }
                bipartite_match_poles_draw_options = {
                    "ax": ax[1][0], 
                    "pos": self.new_bipartite_match_poles_pos, 
                    "nodelist": self.new_selected_pole_index, 
                    "node_size": 10 * self.DRAW_NODE_SIZE, 
                    "node_color": "green", 
                    "node_shape": '^', 
                    "label": "Assigned Pole Node (place DAP)"
                }
                bipartite_common_edges_draw_options = {
                    "ax": ax[1][0], 
                    "pos": dict(**self.new_bipartite_cluster_centers_pos, **self.new_bipartite_poles_pos), 
                    "edgelist": self.new_bipartite_center_pole_edges, 
                    "width": self.DRAW_EDGE_WIDTH, 
                    "edge_color": "grey", 
                    "arrows": False
                }
                bipartite_match_edges_draw_options = {
                    "ax": ax[1][0], 
                    "pos": dict(**self.new_bipartite_cluster_centers_pos, **self.new_bipartite_match_poles_pos), 
                    "edgelist": self.new_bitpartite_match_edges, 
                    "width": 30 * self.DRAW_EDGE_WIDTH, 
                    "edge_color": "blue", 
                    "arrowsize": 100 * self.DRAW_EDGE_WIDTH, 
                    "arrowstyle": '->'
                }
                
                ax[1][0].cla()
                # Draw
                nx.draw_networkx_nodes(self.B, **bipartite_cluster_centers_draw_options)
                nx.draw_networkx_nodes(self.B, **bipartite_poles_draw_options)
                nx.draw_networkx_nodes(self.B, **bipartite_match_poles_draw_options)
                nx.draw_networkx_edges(self.B, **bipartite_common_edges_draw_options)
                nx.draw_networkx_edges(self.B, **bipartite_match_edges_draw_options)
                
                # Draw Settings
                ax[1][0].legend(loc = 'lower right')
                ax[1][0].set_title("Assign Cluster Centers Bipartite")
                ax[1][0].text(0.01, 0.95, "K = " + str(self.K), transform=ax[1][0].transAxes, 
                              fontdict={'size': '10', 'color': 'black', 'weight': 'bold'})
                
                plt.pause(0.1)
            
            if self.DRAW_CONTROL:
                
                # Set drawing options for the graph
                selected_pole_nodes_draw_options = {
                    "ax": ax[0][1], 
                    "pos": self.new_selected_pole_pos, 
                    "nodelist": self.new_selected_pole_index, 
                    "node_size": 20 * self.DRAW_NODE_SIZE, 
                    "node_color": "yellow", 
                    "node_shape": '^', 
                    "label": "Selected Poles"
                }
                
                # Draw
                nx.draw_networkx_nodes(self.G, **selected_pole_nodes_draw_options)

                # Draw Settings
                ax[0][1].legend(loc = 'lower right')
                ax[0][1].set_title("Heuristic Approach Visualization")
                ax[0][1].text(0.01, 0.95, "K = " + str(self.K), transform=ax[0][1].transAxes, 
                              fontdict={'size': '10', 'color': 'black', 'weight': 'bold'})
                ax[0][1].tick_params(left = True, bottom = True, labelleft = True, labelbottom = True)
                
                plt.pause(0.1)
            
            # Step3: find routing of clusters
            # ergodic clusters
            self.new_routing_cost = 0
            self.new_routing.clear()
            for index, s in enumerate(self.new_cluster_centers_index):
                
                subG_sm_nodes_index    = []
                subG_sm_nodes_pos_list = []
                
                for sm_node_index in self.G.neighbors(s):
                    sm_node_pos = self.sm_nodes_pos[sm_node_index]
                    subG_sm_nodes_index.append(sm_node_index)
                    subG_sm_nodes_pos_list.append(sm_node_pos)
                    
                t = mwmresult[s]
                subG = self.G.subgraph(subG_sm_nodes_index + [s, t])
                
                dis_sm   = spatial.distance_matrix(subG_sm_nodes_pos_list, subG_sm_nodes_pos_list)
                dis_sm_t = spatial.distance_matrix(subG_sm_nodes_pos_list, [self.new_selected_pole_pos[t]])
                
                # get filted distance between sm u and sm v or sm u and dap t
                subG_filted_edges_dict = {}
                for i, sm_u in enumerate(subG_sm_nodes_index):
                    d_ut = (dis_sm_t[[i], [0]])[0]
                    if d_ut <= self.PARAM_D_MAX:
                        subG_filted_edges_dict[sm_u, t] = self.PARAM_b * self.PL(d_ut)
                        for j, sm_v in enumerate(subG_sm_nodes_index):
                            d_uv = (dis_sm[[i], [j]])[0]
                            if j != i and d_uv <= self.PARAM_D_MAX and d_uv < d_ut:
                                subG_filted_edges_dict[sm_u, sm_v] = self.PARAM_b * self.PL(d_uv) + self.PARAM_c
                    else:
                        for j, sm_v in enumerate(subG_sm_nodes_index):
                            d_uv = (dis_sm[[i], [j]])[0]
                            if j != i and d_uv <= self.PARAM_D_MAX:
                                subG_filted_edges_dict[sm_u, sm_v] = self.PARAM_b * self.PL(d_uv) + self.PARAM_c
                                
                for edge in subG.edges():
                    subG_filted_edges_dict[edge] = 0
                    
                subG_edges, Beta = gp.multidict(subG_filted_edges_dict)
                
                self.G.remove_edges_from(subG_edges.select(s, '*'))
                self.G.add_edges_from(subG_edges)
                
                if self.DRAW_CONTROL:
                    
                    # Set drawing options for the graph
                    G_filted_edges_draw_options = {
                        "ax":            ax[0][1], 
                        "pos":           dict(**self.sm_nodes_pos, **self.cluster_centers_pos, **self.new_selected_pole_pos), 
                        "width":         2 * self.DRAW_EDGE_WIDTH, 
                        "edge_color":    "black", 
                        "style":         'solid', 
                        "arrows":        False, 
                        "label":         "filted edges"
                    }
                    
                    ax[0][1].cla()
                    # Set canvas and axis
                    ax[0][1].set_xlim([-self.AREA_RANGE/2 - self.AREA_RANGE/30, self.AREA_RANGE/2 + self.AREA_RANGE/30])
                    ax[0][1].set_ylim([-self.AREA_RANGE/2 - self.AREA_RANGE/30, self.AREA_RANGE/2 + self.AREA_RANGE/30])

                    # Draw
                    nx.draw_networkx_nodes(self.G, **cluster_sm_nodes_draw_options)
                    nx.draw_networkx_nodes(self.G, **cluster_centers_draw_options)
                    nx.draw_networkx_nodes(self.G, **selected_pole_nodes_draw_options)
                    nx.draw_networkx_edges(self.G, **G_filted_edges_draw_options)


                    # Draw Settings
                    ax[0][1].legend(loc = 'lower right')
                    ax[0][1].set_title("Heuristic Approach Visualization")
                    ax[0][1].text(0.01, 0.95, "K = " + str(self.K), transform=ax[0][1].transAxes, 
                              fontdict={'size': '10', 'color': 'black', 'weight': 'bold'})
                    ax[0][1].tick_params(left = True, bottom = True, labelleft = True, labelbottom = True)

                    plt.pause(0.1)
                
                cur_routing, cur_routing_cost = self.ILP(subG_edges, Beta, subG_sm_nodes_index, s, t)
                
                if self.RETURN_STEP1:
                    break
                else:
                    self.new_routing_cost += cur_routing_cost
                    self.new_routing.update(cur_routing)
                    
            if self.RETURN_STEP1:
                self.RETURN_STEP1 = False
                self.G.clear_edges()
                self.G.remove_nodes_from(self.new_cluster_centers_index)
                continue
                
            self.new_installation_cost = self.PARAM_a * self.K
            self.new_cost              = self.new_installation_cost + self.new_routing_cost
            
            
            
            if self.new_cost - self.old_cost > self.tol:
                print("[DESCRIBE] " + "The best routing and lowest cost have been found.", "The minimum cost is " + str(self.old_cost) + ".")
                break
            else:
                print("[OBJECTIVE] " + "old cost: " + str(self.old_cost) + ", new cost: " + str(self.new_cost))
                print("[DESCRIBE] " + "Optimal route and lowest cost not found.")
                
                self.K += 1
                self.Iter += 1
                self.old_cost                          = self.new_cost
                self.old_installation_cost             = self.new_installation_cost
                self.old_routing_cost                  = self.new_routing_cost
                self.old_routing                       = copy.deepcopy(self.new_routing)
                self.old_bipartite_center_pole_edges   = copy.deepcopy(self.new_bipartite_center_pole_edges)
                self.old_selected_pole_index           = copy.deepcopy(self.new_selected_pole_index)
                self.old_selected_pole_pos             = copy.deepcopy(self.new_selected_pole_pos)
                self.old_cluster_centers_index         = copy.deepcopy(self.new_cluster_centers_index)
                
                if self.DRAW_CONTROL:
                    self.old_cluster_sm_nodes_color        = copy.deepcopy(self.new_cluster_sm_nodes_color)
                    self.old_bipartite_cluster_centers_pos = copy.deepcopy(self.new_bipartite_cluster_centers_pos)
                    self.old_bipartite_poles_pos           = copy.deepcopy(self.new_bipartite_poles_pos)
                    self.old_bipartite_match_poles_pos     = copy.deepcopy(self.new_bipartite_match_poles_pos)
                    self.old_bitpartite_match_edges        = copy.deepcopy(self.new_bitpartite_match_edges)
                
                self.G.clear_edges()
                self.G.remove_nodes_from(self.new_cluster_centers_index)
                
                self.Iter_index.append(self.Iter)
                
                self.Iter_installation_cost.append(self.new_installation_cost)
                
                delay_cost = 0
                for edge in self.new_routing.keys():
                    if edge[0] in self.new_cluster_centers_index or edge[1] in self.new_selected_pole_index:
                        pass
                    else:
                        delay_cost += self.PARAM_c * self.new_routing[edge]

                transmission_cost = self.new_routing_cost - delay_cost
                
                self.Iter_transmission_cost.append(transmission_cost)
                
                self.Iter_delay_cost.append(delay_cost)
                
                if self.DRAW_CONTROL:
                    
                    ax[1][1].cla()
                    
                    bottom_delay_cost = []
                    for i in range(len(self.Iter_transmission_cost)):
                        sum = self.Iter_transmission_cost[i] + self.Iter_installation_cost[i]
                        bottom_delay_cost.append(sum)
                    
                    ax[1][1].bar(self.Iter_index, self.Iter_installation_cost, width = 0.35, 
                                 color = 'blue', label = "Installation Cost")
                    ax[1][1].bar(self.Iter_index, self.Iter_transmission_cost, 
                                 bottom = self.Iter_installation_cost, width = 0.35, color = 'orange', label = "Transmission Cost")
                    ax[1][1].bar(self.Iter_index, self.Iter_delay_cost, 
                                 bottom = bottom_delay_cost, width = 0.35, color = 'green', label = "Delay Cost")
                    
                    for index, it in enumerate(self.Iter_index):
                        ax[1][1].text(it - 0.10, self.Iter_installation_cost[index] + self.Iter_transmission_cost[index] + self.Iter_delay_cost[index], 
                                      str(int(self.Iter_installation_cost[index] + self.Iter_transmission_cost[index] + self.Iter_delay_cost[index])))
                    
                    ax[1][1].legend(loc = 'lower right')
                    ax[1][1].set_title("Cost Optimization and Contributions")
                    ax[1][1].set_xlabel("Iterations / times")
                    ax[1][1].set_xlim([0, self.Iter + 1])
                    
                    plt.pause(0.1)
                    
        print("[Result]:", 
              "Installation Cost:", self.Iter_installation_cost[-1], 
              "Transmission Cost:", self.Iter_transmission_cost[-1], 
              "Delay Cost", self.Iter_delay_cost[-1], 
              "Total Cost", self.old_cost)
                
        if self.DRAW_CONTROL:
            
            self.G.clear_edges()
            
            routing_edge = []
            for edge in self.old_routing.keys():
                w_package = self.old_routing[edge]
                if w_package != 0:
                    routing_edge.append((edge[0], edge[1], {'width': w_package}))

            self.G.add_edges_from(routing_edge)
            self.G.remove_nodes_from(self.old_cluster_centers_index)    
            
            # Set drawing options for the graph
            G_cluster_sm_nodes_draw_options = {
                "ax":            ax[0][1], 
                "pos":           self.sm_nodes_pos, 
                "nodelist":      self.sm_nodes_index, 
                "node_size":     self.DRAW_NODE_SIZE, 
                "node_color":    self.old_cluster_sm_nodes_color, 
                "node_shape":    'o', 
                "label":         "Smart Meters of Different Clusters"
            }
            G_selected_pole_nodes_draw_options = {
                "ax":            ax[0][1], 
                "pos":           self.old_selected_pole_pos, 
                "nodelist":      self.old_selected_pole_index, 
                "node_size":     20 * self.DRAW_NODE_SIZE, 
                "node_color":    "yellow", 
                "node_shape":    '^', 
                "label":         "Selected Poles"
            }
            G_routing_edges_draw_options = {
                "ax":            ax[0][1], 
                "pos":           dict(**self.sm_nodes_pos, **self.pole_nodes_pos), 
                "width":         [w[2] * 10 * self.DRAW_EDGE_WIDTH for w in self.G.edges.data('width')], 
                "edge_color":    "black", 
                "style":         'solid', 
                "arrows":        False, 
                "label":         "routing edges"
            }
            
            ax[0][1].cla()
            # Set canvas and axis
            ax[0][1].set_xlim([-self.AREA_RANGE/2 - self.AREA_RANGE/30, self.AREA_RANGE/2 + self.AREA_RANGE/30])
            ax[0][1].set_ylim([-self.AREA_RANGE/2 - self.AREA_RANGE/30, self.AREA_RANGE/2 + self.AREA_RANGE/30])

            # Draw
            nx.draw_networkx_nodes(self.G, **G_cluster_sm_nodes_draw_options)
            nx.draw_networkx_nodes(self.G, **G_selected_pole_nodes_draw_options)
            nx.draw_networkx_edges(self.G, **G_routing_edges_draw_options)

            # Draw Settings
            ax[0][1].legend(loc = 'lower right')
            ax[0][1].set_title("Heuristic Approach Result")
            ax[0][1].text(0.01, 0.95, "K = " + str(self.K - 1), transform=ax[0][1].transAxes, 
                          fontdict={'size': '10', 'color': 'black', 'weight': 'bold'})
            ax[0][1].tick_params(left = True, bottom = True, labelleft = True, labelbottom = True)

            plt.pause(0.1)
            
        if self.DRAW_CONTROL:
            
            self.B.clear()
            
            self.B.add_nodes_from(self.old_cluster_centers_index, bipartite = 0)
            self.B.add_nodes_from(self.pole_nodes_index, bipartite = 1)
            self.B.add_edges_from(self.old_bipartite_center_pole_edges)
            
            B_bipartite_cluster_centers_draw_options = {
                "ax": ax[1][0], 
                "pos": self.old_bipartite_cluster_centers_pos, 
                "nodelist": self.old_cluster_centers_index, 
                "node_size": 10 * self.DRAW_NODE_SIZE, 
                "node_color": "red", 
                "node_shape": 'o', 
                "label": "Clustering Center Node"
            }
            B_bipartite_poles_draw_options = {
                "ax": ax[1][0], 
                "pos": self.old_bipartite_poles_pos, 
                "nodelist": self.pole_nodes_index, 
                "node_size": self.DRAW_NODE_SIZE, 
                "node_color": "red", 
                "node_shape": '^', 
                "label": "Pole Node"
            }
            B_bipartite_match_poles_draw_options = {
                "ax": ax[1][0], 
                "pos": self.old_bipartite_match_poles_pos, 
                "nodelist": self.old_selected_pole_index, 
                "node_size": 10 * self.DRAW_NODE_SIZE, 
                "node_color": "green", 
                "node_shape": '^', 
                "label": "Assigned Pole Node (place DAP)"
            }
            B_bipartite_common_edges_draw_options = {
                "ax": ax[1][0], 
                "pos": dict(**self.old_bipartite_cluster_centers_pos, **self.old_bipartite_poles_pos), 
                "edgelist": self.old_bipartite_center_pole_edges, 
                "width": self.DRAW_EDGE_WIDTH, 
                "edge_color": "grey", 
                "arrows": False
            }
            B_bipartite_match_edges_draw_options = {
                "ax": ax[1][0], 
                "pos": dict(**self.old_bipartite_cluster_centers_pos, **self.old_bipartite_match_poles_pos), 
                "edgelist": self.old_bitpartite_match_edges, 
                "width": 30 * self.DRAW_EDGE_WIDTH, 
                "edge_color": "blue", 
                "arrowsize": 100 * self.DRAW_EDGE_WIDTH, 
                "arrowstyle": '->'
            }
            
            ax[1][0].cla()
            # Draw
            nx.draw_networkx_nodes(self.B, **B_bipartite_cluster_centers_draw_options)
            nx.draw_networkx_nodes(self.B, **B_bipartite_poles_draw_options)
            nx.draw_networkx_nodes(self.B, **B_bipartite_match_poles_draw_options)
            nx.draw_networkx_edges(self.B, **B_bipartite_common_edges_draw_options)
            nx.draw_networkx_edges(self.B, **B_bipartite_match_edges_draw_options)

            # Draw Settings
            ax[1][0].legend(loc = 'lower right')
            ax[1][0].set_title("Assign Cluster Centers Bipartite")
            ax[1][0].text(0.01, 0.95, "K = " + str(self.K - 1), transform=ax[1][0].transAxes, 
                          fontdict={'size': '10', 'color': 'black', 'weight': 'bold'})
            
            plt.pause(0.1)
            
        self.ClearILPEnv()

##### Compare the effects of different clustering algorithms on the heuristic algorithms

In [ ]:
ha = HeuristicApproach(N_MAX_SS=10, D_MAX=150, c=5, DRAW_CONTROL=True, ILP_INFO=False)
ha.GetSimData()

In [ ]:
ha.InitializeCleanEnvironment()
ha.OurHeuristic(clustering_type = 'kmeans', custom_metric = 'eu')

In [ ]:
ha.InitializeCleanEnvironment()
ha.OurHeuristic(clustering_type = 'minibatchkmeans', custom_metric = 'eu')

In [ ]:
ha.InitializeCleanEnvironment()
ha.OurHeuristic(clustering_type = 'kmedoids', custom_metric = 'eu')

In [ ]:
ha.InitializeCleanEnvironment()
ha.OurHeuristic(clustering_type = 'kmedoids', custom_metric = 'en')

In [ ]:
ha.InitializeCleanEnvironment()
ha.OurHeuristic(clustering_type = 'kmedoids', custom_metric = 'ct')

In [ ]:
ha.InitializeCleanEnvironment()
ha.OurHeuristic(clustering_type = 'kmedoids', custom_metric = 'cte')

In [ ]:
ha.InitializeCleanEnvironment()
ha.OurHeuristic(clustering_type = 'spectral', custom_metric = 'eu')

In [ ]:
ha.InitializeCleanEnvironment()
ha.OurHeuristic(clustering_type = 'spectral', custom_metric = 'en')

In [ ]:
ha.InitializeCleanEnvironment()
ha.OurHeuristic(clustering_type = 'spectral', custom_metric = 'ct')

In [ ]:
ha.InitializeCleanEnvironment()
ha.OurHeuristic(clustering_type = 'spectral', custom_metric = 'cte')

In [ ]:
ha.InitializeCleanEnvironment()
ha.OurHeuristic(clustering_type = 'agglomerative', custom_metric = 'eu')

In [ ]:
ha.InitializeCleanEnvironment()
ha.OurHeuristic(clustering_type = 'agglomerative', custom_metric = 'en')

In [ ]:
ha.InitializeCleanEnvironment()
ha.OurHeuristic(clustering_type = 'agglomerative', custom_metric = 'ct')

In [ ]:
ha.InitializeCleanEnvironment()
ha.OurHeuristic(clustering_type = 'agglomerative', custom_metric = 'cte')

##### Compare the effects of different parameters on the heuristic algorithm

In [ ]:
ha_nmaxss = HeuristicApproach(DRAW_CONTROL=True, ILP_INFO=False)
ha_nmaxss.GetSimData()

In [ ]:
ha_nmaxss.InitializeCleanEnvironment()
ha_nmaxss.ManualChangeParameter(N_MAX_SS = 3, D_MAX = 150, c = 5, alpha = 3)
ha_nmaxss.OurHeuristic(clustering_type = 'kmeans', custom_metric = 'eu')

In [ ]:
ha_nmaxss.InitializeCleanEnvironment()
ha_nmaxss.ManualChangeParameter(N_MAX_SS = 10, D_MAX = 150, c = 5, alpha = 3)
ha_nmaxss.OurHeuristic(clustering_type = 'kmeans', custom_metric = 'eu')

In [ ]:
ha_nmaxss.InitializeCleanEnvironment()
ha_nmaxss.ManualChangeParameter(N_MAX_SS = 10, D_MAX = 100, c = 0.5, alpha = 3)
ha_nmaxss.OurHeuristic(clustering_type = 'kmeans', custom_metric = 'eu')

In [ ]:
ha_nmaxss.InitializeCleanEnvironment()
ha_nmaxss.ManualChangeParameter(N_MAX_SS = 10, D_MAX = 100, c = 5, alpha = 3)
ha_nmaxss.OurHeuristic(clustering_type = 'kmeans', custom_metric = 'eu')

In [ ]:
ha_nmaxss.InitializeCleanEnvironment()
ha_nmaxss.ManualChangeParameter(N_MAX_SS = 10, D_MAX = 100, c = 50, alpha = 3)
ha_nmaxss.OurHeuristic(clustering_type = 'kmeans', custom_metric = 'eu')

In [ ]:
ha_nmaxss.InitializeCleanEnvironment()
ha_nmaxss.ManualChangeParameter(N_MAX_SS = 10, D_MAX = 100, c = 5, alpha = 2)
ha_nmaxss.OurHeuristic(clustering_type = 'kmeans', custom_metric = 'eu')

In [ ]:
ha_nmaxss.InitializeCleanEnvironment()
ha_nmaxss.ManualChangeParameter(N_MAX_SS = 10, D_MAX = 100, c = 5, alpha = 4)
ha_nmaxss.OurHeuristic(clustering_type = 'kmeans', custom_metric = 'eu')